<a href="https://colab.research.google.com/github/parimalakettymuthu/MachineLearning-Projects/blob/main/Multiclass_Classification_assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import sys 
sys.path.append('/content/drive/MyDrive/NLP/custom-functions')

In [15]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  !pip install -U swifter -qq
  !pip install -U nltk -qq
  !pip install -U spacy -qq
  !python -m spacy download en_core_web_sm -qq
  !pip install torchtext --upgrade
  base_path = '/content/drive/MyDrive/NLP/assignment5'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-31 22:08:57.106503: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-31 22:08:59.403703: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-03-31 22:09:02.555510: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 42.7 MB/s eta 0:00:00
✔ Download and installation successf

In [16]:
from pathlib import Path
import pandas as pd
import torch
import torch.nn as inn
from torchtext.vocab import vocab
from collections import Counter
from torch.utils.data import Dataset, DataLoader


In [17]:
data_folder = Path(base_path)
for entries in data_folder.iterdir():
  print(entries.name)

multiclass_hw_cleaned.csv
Multiclass_Classification_assignment5.ipynb


In [18]:
stackExchange1 = pd.read_csv(data_folder/ 'multiclass_hw_cleaned.csv', encoding = 'ISO-8859-1')
print(stackExchange1.shape)
print(stackExchange1.head(2))

(188878, 6)
   Unnamed: 0                                     Title  \
0           0   detail disclosure indicator on UIButton   
1           1  hello world fails to show up in emulator   

                                                Body  \
0  <p>Is there a simple way to place a detail dis...   
1  <p>I followed Hello World tutorial exactly.  E...   

                                        cleaned_text     Tags  \
0  detail disclosure indicator uibutton simple wa...   iphone   
1  hello world fail emulator follow hello world t...  android   

   Tag_Number_final  
0                 8  
1                 4  


In [19]:
stackExchange = pd.DataFrame().assign(cleaned_text=stackExchange1['cleaned_text'], Tag_Number_final=stackExchange1['Tag_Number_final'])
print(stackExchange)

                                             cleaned_text  Tag_Number_final
0       detail disclosure indicator uibutton simple wa...                 8
1       hello world fail emulator follow hello world t...                 4
2       jshint throw possible strict violation line tr...                 3
3       programmatically bound column invisible try da...                 9
4       edittext get focus soft keyboard android home ...                 4
...                                                   ...               ...
188873  reload tab activity tab change reload activity...                 4
188874  learning index loop list contain list inner li...                 7
188875  module pattern | private access pass reference...                 3
188876  listener homebutton android   possible duplica...                 4
188877  php optimize multiple conditional loop multipl...                 2

[188878 rows x 2 columns]


In [20]:
stackExchange.dropna(inplace=True)
print(stackExchange)

                                             cleaned_text  Tag_Number_final
0       detail disclosure indicator uibutton simple wa...                 8
1       hello world fail emulator follow hello world t...                 4
2       jshint throw possible strict violation line tr...                 3
3       programmatically bound column invisible try da...                 9
4       edittext get focus soft keyboard android home ...                 4
...                                                   ...               ...
188873  reload tab activity tab change reload activity...                 4
188874  learning index loop list contain list inner li...                 7
188875  module pattern | private access pass reference...                 3
188876  listener homebutton android   possible duplica...                 4
188877  php optimize multiple conditional loop multipl...                 2

[188874 rows x 2 columns]


In [21]:
from sklearn.model_selection import train_test_split

# Split data into train and test sets (80/20 split)
SExchangeTrain, test_val = train_test_split(stackExchange, test_size=0.2, random_state=42)

# Split train set into train and validation sets (50/50 split)
SExchangeTest, SExchangeVal = train_test_split(test_val, test_size=0.5, random_state=42)

# Print the shapes & percentage of split of the resulting datasets
print("Train set shape: ", SExchangeTrain.shape, "Split %: ", len(SExchangeTrain)/len(stackExchange))
print("Validation set shape: ", SExchangeVal.shape,"Split %: ", len(SExchangeVal)/len(stackExchange))
print("Test set shape: ", SExchangeTest.shape, "Split %: ", len(SExchangeTest)/len(stackExchange))

Train set shape:  (151099, 2) Split %:  0.7999989410930038
Validation set shape:  (18888, 2) Split %:  0.1000031767209886
Test set shape:  (18887, 2) Split %:  0.09999788218600761


In [22]:
X_SExchangeTrain = SExchangeTrain["cleaned_text"].values
X_SExchangeTest = SExchangeTest["cleaned_text"].values
X_SExchangeValid = SExchangeVal["cleaned_text"].values

Y_SExchangeTrain = SExchangeTrain["Tag_Number_final"].values
Y_SExchangeTest = SExchangeTest["Tag_Number_final"].values
Y_SExchangeValid = SExchangeVal["Tag_Number_final"].values

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
!pip install wandb -qq
import wandb
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 KB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 3.8 MB/s eta 0:00:00


# Create Vectors using Tfidf

In [24]:
#vectorizer = TfidfVectorizer(
 #   stop_words='english', min_df=4, max_features=5000).fit(X_SExchangeTrain)
#X_SEtrain_vec = vectorizer.transform(X_SExchangeTrain)
#X_SEvalid_vec = vectorizer.transform(X_SExchangeValid)
#X_SEtest_vec = vectorizer.transform(X_SExchangeTest)

#Create PyTorch Tensors

In [25]:
# Creating a tensors of data
#X_SEtrain_Tensor = torch.tensor(X_SEtrain_vec.toarray()).float()
#X_SEvalid_Tensor = torch.tensor(X_SEvalid_vec.toarray()).float()
#X_SEtest_Tensor = torch.tensor(X_SEtest_vec.toarray()).float()

#y_SEtrain_Tensor = torch.tensor(np.array(Y_SExchangeTrain)).long()
#y_SEvalid_Tensor = torch.tensor(np.array(Y_SExchangeValid)).long()
#y_SEtest_Tensor = torch.tensor(np.array(Y_SExchangeTest)).long()


#Create PyTorch Dataset

In [26]:
# Creating tensor dataset
#SETrain_set = torch.utils.data.TensorDataset(X_SEtrain_Tensor, y_SEtrain_Tensor)
#SETest_set = torch.utils.data.TensorDataset(X_SEtest_Tensor, y_SEtest_Tensor)
#SEValid_set = torch.utils.data.TensorDataset(X_SEvalid_Tensor, y_SEvalid_Tensor)

#Custom class

In [31]:
class CustomClass(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y 

  def __getitem__(self, idx):
    text = self.X.iloc(idx)
    labels = self.y.iloc(idx)
    tup = (text, labels)
    return tup

In [ ]:
tran_dataset = customClass(X_SExchangeTrain, Y_SExchangeTrain)

#Create Vocab

In [ ]:
#def createVocab(,):

  #return 

#Create DataLoaders

In [ ]:
#Initiale the batch size 
#batch_size = 256

#Trainset dataloader
#SETrain_Loader = torch.utils.data.DataLoader(dataset=SETrain_set,
                                           #batch_size=batch_size,
                                           #shuffle=True)
#Testset dataloader
#SETest_Loader = torch.utils.data.DataLoader(dataset=SETest_set,
                                          # batch_size=batch_size,
                                           #shuffle=True)
#Validset dataloader
#SEValid_Loader = torch.utils.data.DataLoader(dataset=SEValid_set,
                                           #batch_size=batch_size,
                                           #shuffle=True)
